In [8]:
# importing required libs
from fatsecret import Fatsecret
import boto3
from credentials import customerKey, customerSecret, awsKey, awsSecret
from credentials import user, password, host, database
from datetime import timedelta
import datetime
from dateutil.relativedelta import relativedelta
import json
import os
import mysql.connector

In [9]:
# connecting to the API and getting authenticated
fs = Fatsecret(customerKey, customerSecret)
print(fs.get_authorize_url())
AccessCode = input("Enter the access code: ")
fs.authenticate(AccessCode)

https://www.fatsecret.com/oauth/authorize?oauth_token=868752d497f94cd898d7de3bb014bc18
Enter the access code: 4452370


('828e31def45e4bf29cd43d3d88c5b5d7', '3b00bbf54f57467ab45843ad8443c014')

In [10]:
def returnDates(user, password, host, database):
    database = mysql.connector.connect(user = user, password = password, host = host, database = database)
    cursor = database.cursor()
    cursor.execute('''SELECT MAX(date_of_log)
                        FROM food_entry;''')
    endDate = cursor.fetchall()[0][0]
    yesterDay = datetime.datetime.now().date() - timedelta(days=1)
    difference = yesterDay - endDate
    days = []
    if difference.days == 0: 
        print("No loading needed")
        return None
    elif difference.days <= 10:
        while endDate != yesterDay:
            endDate += timedelta(days=1)
            days.append(endDate.strftime('%Y-%m-%d'))
    else: 
        tenMore = endDate + timedelta(days = 10)
        while endDate <= tenMore:
            endDate += timedelta(days = 1)
            days.append(endDate.strftime('%Y-%m-%d'))
    return days

In [11]:
class infoGetter: 
    def __init__(self, fs):
        self.fs = fs
    
    def getProfile(self):
        return self.fs.profile_get()
    
    def getFoodEntries(self, days):
        allFoodEntries = []
        for day in days:
            d = datetime.datetime.strptime(day, '%Y-%m-%d')
            oneDayFoodEntry = fs.food_entries_get(date = d)
            allFoodEntries.extend(oneDayFoodEntry)
        return allFoodEntries
    
    def caloriesBurnedDay(self, months, year):
        dates = []
        for month in months: 
            dates.append(datetime.datetime(year, month, 1))
        caloriesBurned = []
        for date in dates:
            caloriesMonth = fs.exercise_entries_get_month(date = date)
            if type(caloriesMonth) == dict:
                caloriesBurned.append(caloriesMonth)
            elif type(caloriesMonth) == list:
                caloriesBurned.extend(caloriesMonth)
        return caloriesBurned
    
    def getWeights(self, months, year):
        weights = []
        for month in months:
            date = f'{year}-{month}-1'
            weightMonth = fs.weights_get_month(date = datetime.datetime.strptime(date, '%Y-%m-%d'))
            weights.extend(weightMonth)
        return weights
    
    def getBMR(self, months, year, man, age):
        profileDict = self.getProfile()
        weights = self.getWeights(months, year)
        for weight in weights:
            if man == 1:
                weight["BMR"] = 88.362 + (13.397 * float(weight["weight_kg"])) + (4.799 * float(profileDict["height_cm"])) - (5.677 * age)
            else:
                weight["BMR"] = 447.593 + (9.247 * float(weight["weight_kg"])) + (3.098 * float(profileDict["height_cm"])) - (4.330 * age)
        return weights
    


In [13]:
if __name__ == "__main__":
    days = returnDates(user, password, host, database)
    ig = infoGetter(fs)
    months = list(range(1, 5))
    
    profile = ig.getProfile()
    foodEntries = ig.getFoodEntries(days)
    caloriesBurned = ig.caloriesBurnedDay(months, 2024)
    weights = ig.getBMR(months = months,year = 2024, man = 1, age = 23)
    with open("local_warehouse/profile.json", "w") as file:
        profile["profile_id"] = 1
        json.dump(profile, file)
    
    with open("local_warehouse/caloriesBurned.json", "w") as file: 
        json.dump(caloriesBurned, file)
    
    with open("local_warehouse/weights.json", "w") as file:
        json.dump(weights, file)
        
    with open("local_warehouse/foodEntries.json", "r") as file:
        existing_data = json.load(file)

    existing_data.extend(foodEntries)

    with open("local_warehouse/foodEntries.json", "w") as file:
        json.dump(existing_data, file, indent=4)

In [14]:
class s3Writer:
    def __init__(self, awsKey, awsSecret):
        self.awsKey = awsKey
        self.awsSecret = awsSecret
    
    def signIn(self):
        s3 = boto3.resource('s3', aws_access_key_id = self.awsKey, 
                            aws_secret_access_key = self.awsSecret, region_name='us-east-1')
        return s3
    
    def write(self, buketName, writingDict):
        s3 = self.signIn()
        bucket = s3.Bucket(buketName)

        for json_key in writingDict:
            json_string = json.dumps(writingDict[json_key])
            bucket.put_object(Key=json_key, Body=json_string)

In [15]:
if __name__ == "__main__":
    writingDict = {
        "profile.json": profile,
        "foodEntries.json": existing_data,
        "caloriesBurned.json": caloriesBurned,
        "weights.json": weights
    }

    s3writer = s3Writer(awsKey, awsSecret)
    s3writer.write("fatsecret", writingDict)